In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import random
from scipy.integrate import solve_ivp

def construct_intermediate_hamiltonian(H0,Hf,s):
    return H0 * (1-s) + Hf * s

# Time-dependent Schrödinger equation with normalization
def normalized_tdse(t, psi, H0, Hf, t_total, interpolation_type = "linear"):
    psi = psi / np.linalg.norm(psi)  # Normalize the wavefunction
    
    if interpolation_type == "sine-squared":
        s = np.sin((np.pi/2) * (t / t_total))**2  # sine-squared interpolation
    elif interpolation_type == "linear":
        s = t/t_total # linear interpolation
    elif interpolation_type == "smoothstep":
        s = 3*(t / t_total)**2 - 2*(t / t_total)**3 # cubic (smoothstep) interpolation
    
    H_s = construct_intermediate_hamiltonian(H0, Hf, s)
    return -1j * H_s.dot(psi)

# Function to evolve the system
def evolve_system(H0, Hf, psi0, t_total, t_points, interpolation_type = "sine-squared"):
    t_eval = np.linspace(0, t_total, t_points)
    sol = solve_ivp(
        normalized_tdse, [0, t_total], psi0, t_eval=t_eval, 
        args=(H0, Hf, t_total, interpolation_type), method="RK45",
        rtol=1e-10, atol=1e-12
    )
    return sol.t, sol.y

# Function to calculate the instantaneous energy
def instantaneous_energy(H, psi):
    return np.real(np.vdot(psi, H @ psi))

In [5]:
# Parameters
N = 2
M = 2
mu = 100
V = 1

t_total_values = [1000,10000]  # List of t_total values to test
t_points = 100  # Higher resolution for better accuracy

# Prepare the figure
fig, ax = plt.subplots()

cmap = get_cmap("gist_rainbow", len(t_total_values))

for i, t_total in enumerate(t_total_values):
    color = cmap(i)
    # Random J/V ratio for each t_total
    J_V_ratio = 0.2
    J = J_V_ratio * V

    # Construct Hamiltonians
    H0 = construct_initial_hamiltonian(N, M, mu)
    Hf = construct_hamiltonian(N, M, J, V)

    # Initial state
    eigvals, eigvecs = np.linalg.eigh(H0)
    psi0 = eigvecs[:, 0]
    psi0 = psi0 / np.linalg.norm(psi0)  # Ensure normalization

    # Evolve the system
    t, psi_t = evolve_system(H0, Hf, psi0, t_total, t_points)

    # Calculate ground state probabilities
    ground_state_prob = np.zeros_like(t)
    for i in range(len(t)):
        s = np.sin((np.pi/2) * (t[i] / t_total))**2
        H_s = construct_intermediate_hamiltonian(H0, Hf, s)
        _, eigvecs_s = exact_diagonalize(H_s)
        psi_ground_state_s = eigvecs_s[0]  # Ground state at time t
        ground_state_prob[i] = np.abs(np.dot(np.conjugate(psi_ground_state_s), psi_t[:, i]))**2

    # Plot the result for this t_total
    ax.plot(t/t_total, ground_state_prob, '.', color = color, label=f"$t_{{total}}$ = {t_total}, J/V = {round(J_V_ratio, 3)}")

# Customize the plot
ax.set_xlabel("Time [t/t_total]")
ax.set_ylabel("Ground State Probability")
ax.set_title(f"Adiabatic Evolution: N = {N}, M = {M}, J/V = {J_V_ratio}")
ax.set_ylim(0, 1.05)
ax.grid()
ax.legend()

# Show the plot
plt.show()

C:\Users\Joe\AppData\Local\Temp\ipykernel_6320\2452537003.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = get_cmap("gist_rainbow", len(t_total_values))
